## Pumpkin Varieties and Color

Load up required libraries and dataset. Convert the data to a dataframe containing a subset of the data: 

Let's look at the relationship between color and variety

In [63]:
import pandas as pd
import numpy as np
import sklearn


full_pumpkins = pd.read_csv('../data/US-pumpkins.csv')

full_pumpkins.head()


,City Name,Type,Package,Variety,Sub Variety,Grade,Date,Low Price,High Price,Mostly Low,...,Unit of Sale,Quality,Condition,Appearance,Storage,Crop,Repack,Trans Mode,Unnamed: 24,Unnamed: 25
0,BALTIMORE,NaN,24 inch bins,NaN,NaN,NaN,4/29/17,270.0,280.0,270.0,...,NaN,NaN,NaN,NaN,NaN,NaN,E,NaN,NaN,NaN
1,BALTIMORE,NaN,24 inch bins,NaN,NaN,NaN,5/6/17,270.0,280.0,270.0,...,NaN,NaN,NaN,NaN,NaN,NaN,E,NaN,NaN,NaN
2,BALTIMORE,NaN,24 inch bins,HOWDEN TYPE,NaN,NaN,9/24/16,160.0,160.0,160.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
3,BALTIMORE,NaN,24 inch bins,HOWDEN TYPE,NaN,NaN,9/24/16,160.0,160.0,160.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
4,BALTIMORE,NaN,24 inch bins,HOWDEN TYPE,NaN,NaN,11/5/16,90.0,100.0,90.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN


In [64]:
#Contando valores nulos en cada columna

print(full_pumpkins.shape)
full_pumpkins.isnull().sum()

(1757, 26)


City Name             0
Type               1712
Package               0
Variety               5
Sub Variety        1461
Grade              1757
Date                  0
Low Price             0
High Price            0
Mostly Low          103
Mostly High         103
Origin                3
Origin District    1626
Item Size           279
Color               616
Environment        1757
Unit of Sale       1595
Quality            1757
Condition          1757
Appearance         1757
Storage            1757
Crop               1757
Repack                0
Trans Mode         1757
Unnamed: 24        1757
Unnamed: 25        1654
dtype: int64

In [65]:
#Eliminando las columnas con menos de 1000 valores no-nulos y eliminando los registros nulos en las columnas restantes

clean_pumpkins = full_pumpkins.dropna(axis=1, thresh=1000)
clean_pumpkins = clean_pumpkins.dropna()
clean_pumpkins.head()

,City Name,Package,Variety,Date,Low Price,High Price,Mostly Low,Mostly High,Origin,Item Size,Color,Repack
2,BALTIMORE,24 inch bins,HOWDEN TYPE,9/24/16,160.0,160.0,160.0,160.0,DELAWARE,med,ORANGE,N
3,BALTIMORE,24 inch bins,HOWDEN TYPE,9/24/16,160.0,160.0,160.0,160.0,VIRGINIA,med,ORANGE,N
4,BALTIMORE,24 inch bins,HOWDEN TYPE,11/5/16,90.0,100.0,90.0,100.0,MARYLAND,lge,ORANGE,N
5,BALTIMORE,24 inch bins,HOWDEN TYPE,11/12/16,90.0,100.0,90.0,100.0,MARYLAND,lge,ORANGE,N
6,BALTIMORE,36 inch bins,HOWDEN TYPE,9/24/16,160.0,170.0,160.0,170.0,MARYLAND,med,ORANGE,N


In [66]:
#Creando una nueva columna con el mes en el que se cosecho cada calabaza y quitando las fechas

month = pd.DatetimeIndex(clean_pumpkins['Date']).month
clean_pumpkins['Month'] = month
clean_pumpkins.drop(axis=1, columns='Date', inplace=True)

#Creando una columna con el precio promedio y quitando las columnas con precios altos y bajos

price = (clean_pumpkins['High Price'] + clean_pumpkins['Low Price']) / 2
clean_pumpkins['Price'] = price
clean_pumpkins.drop(axis=1, columns=['High Price', 'Low Price', "Mostly Low", "Mostly High"], inplace=True)

In [67]:
#Viendo los valores unicos de las columnas no-numericas y no-ordinales que nos quedaron

for column in clean_pumpkins.columns.difference(['Price', 'Month', 'Item Size']):
    print(clean_pumpkins[column].value_counts())


City Name
BOSTON           348
COLUMBIA         173
CHICAGO           87
BALTIMORE         80
DALLAS            60
NEW YORK          58
ATLANTA           35
SAN FRANCISCO     34
PHILADELPHIA      28
LOS ANGELES       26
DETROIT           15
MIAMI              2
Name: count, dtype: int64
Color
ORANGE    795
WHITE     151
Name: count, dtype: int64
Origin
MASSACHUSETTS     194
PENNSYLVANIA      180
MICHIGAN          132
CANADA             70
CALIFORNIA         60
TEXAS              54
MARYLAND           49
OHIO               45
VIRGINIA           43
ILLINOIS           28
NORTH CAROLINA     27
NEW JERSEY         20
NEW YORK           16
DELAWARE           11
VERMONT             6
MEXICO              6
ALABAMA             4
TENNESSEE           1
Name: count, dtype: int64
Package
36 inch bins            490
24 inch bins            268
1/2 bushel cartons      120
1 1/9 bushel cartons     30
bushel cartons           26
1 1/9 bushel crates      11
bins                      1
Name: count, dtype:

In [68]:
#Quitando la columna Repack ya que todos los registros tienen el mismo valor

clean_pumpkins.drop(axis=1, columns='Repack', inplace=True)


In [69]:
#Estandarizando precios para que queden en bushel cartons, segun estimaciones de ChatGPT

clean_pumpkins.loc[clean_pumpkins['Package'] == '36 inch bins', 'Price'] = clean_pumpkins['Price'] * 7.23
clean_pumpkins.loc[clean_pumpkins['Package'] == '24 inch bins', 'Price'] = clean_pumpkins['Price'] * 2.41
clean_pumpkins.loc[clean_pumpkins['Package'] == '1/2 bushel cartons', 'Price'] = clean_pumpkins['Price'] * 0.5
clean_pumpkins.loc[clean_pumpkins['Package'] == '1 1/9 bushel cartons', 'Price'] = clean_pumpkins['Price'] / (10/9)

#Las bushel crates y los bushel cartons tienen el mismo volumen. La diferencia solo esta en la forma y el material de la caja. Por lo que
#se hace la misma conversion para ambos

clean_pumpkins.loc[clean_pumpkins['Package'] == '1 1/9 bushel crates', 'Price'] = clean_pumpkins['Price'] / (10/9)

#Se quita la unica fila con el tipo "bins" ya que faltan detalles y ademas es una categoria con un solo dato
clean_pumpkins = clean_pumpkins[clean_pumpkins['Package'] != 'bins']

clean_pumpkins.head()

KeyError: 'Price Units'

In [ ]:
#Creando un encoder ordinal para la columna Item Size, nuestra unica columna ordinal.

from sklearn.preprocessing import OrdinalEncoder

item_size_categories = [['sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo']]
ordinal_features = ['Item Size']
ordinal_encoder = OrdinalEncoder(categories=item_size_categories)